In [ ]:
#import spark.implicits._
#case class Bank(age: Integer, job: String, marital: String, education: String, balance: Integer)

import re
bank_txt = spark.sparkContext.textFile("/data/examples/bank.csv")

bank_df = bank_txt.map(lambda s: re.sub(r'(?:^\"|\"$)', '', s))\
    .map(lambda s: re.split('\"?;\"?', s))\
    .filter(lambda s: s[0] != "age")\
    .map(lambda s: [int(s[0])] + s[1:4] + [int(s[5])]).toDF(['age', 'job', 'marital', 'education', 'balance'])
bank_df.printSchema()

In [ ]:
bank_df.createOrReplaceTempView('bank_view')
bank_df.limit(10).toPandas()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

em_pdf = spark.sql('select education, marital, count(*) as count from bank_view group by education,marital')\
    .toPandas()
em_pdf.head(3)

In [ ]:
pdf = em_pdf.pivot(index='education', columns='marital', values='count')
pdf.head()

In [ ]:
pdf.plot(kind='bar', stacked=True)
plt.show()

In [ ]:
bank_df = spark.read.csv("/data/examples/bank.csv", header=True, sep=';', 
                      mode="DROPMALFORMED", inferSchema=True)
bank_df.write.mode("overwrite").saveAsTable("default.bank");
bank_df.printSchema()

In [ ]:
bank_df = spark.read.table("bank")
bank_jobs = bank_df.groupBy("job").count()
bank_jobs.createOrReplaceTempView("bank_jobs")
jobs_pd = spark.sql("select * from bank_jobs order by count desc limit 10").toPandas()
jobs_pd.head()

In [ ]:
jobs_pd.plot(kind='bar', x='job', y='count', stacked=True)
plt.show()

In [ ]:
pd.DataFrame(spark.catalog.listTables("default"))

### Show Hive Tables

In [ ]:
spark.sql("show tables").toPandas()